# Curso NVIDIA - Desafio

> Link para o dataset: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents

Etapas do desafio:

1. Carregar o dataset usando o cuDF - como são milhões de registros será bem mais visível a diferença usar o cuDF ao invés de Pandas.
2. Remover os valores nulos
3. Remover colunas desnecessárias
 * Mantenha apenas as seguintes colunas: ['Severity', 'Source', 'County', 'State', 'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']
4. Tratamento com Label encoding
5. Balanceamento do dataset
 * Dica: nessa etapa será necessário trabalhar com a interoperabilidade entre as bibliotecas e então será mais interessante converter para dataframe Pandas, e assim realizar os processamentos necessários. Além disso, será util para a próxima etapa.
6. Tratamento com o encoding - como alternativa, faça o [Ordinal Encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) ao invés do One Hot Encoding.
7. Escalonamento dos valores
8. Divisão do conjunto de treinamento
 * Dica: na sequência, será preciso converter de dataframe Pandas para cuDF
9. Treinar com o algoritmo KNeighborsClassifier
10. Realizar a predição e cálculo de acurácia


In [6]:
!pip install -q kaggle

* Dentro do painel do Kaggle, acesse a página Configurações
https://www.kaggle.com/settings

* Na seção API, selecione o botão [Create New Token]

* Será baixado um arquivo chamado kaggle.json

* Envie esse arquivo para o Colab.


In [7]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
!cp kaggle.json ~/.kaggle/

In [9]:
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets list

ref                                                              title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows                           Netflix Movies and TV Shows                        1MB  2024-04-10 09:48:38           8352        176  1.0              
jaceprater/smokers-health-data                                   Smoker's Health Data                              29KB  2024-04-20 17:09:38           1769         25  1.0              
mexwell/heart-disease-dataset                                    🫀 Heart Disease Dataset                          399KB  2024-04-08 09:43:49           4238         76  1.0              
sujithmandala/second-hand-car-price-prediction                   Secon

In [11]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

us-accidents.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [ ]:
!unzip us-accidents.zip -d dataset

Archive:  us-accidents.zip
replace dataset/US_Accidents_March23.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Instalação e importação

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
import cudf
import cuml
import cupy as cp

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1) Carregamento do dataset

Faremos a leitura do dataset no .csv usando o cuDF

caso tenha problemas de falta de memória ao ler o arquivo, uma opção é limitar as linhas lidas usando o parâmetro nrows na função read_csv. Exemplo: `cudf.read_csv('/content/dataset/US_Accidents_March23.csv', nrows=6_000_000)`

A GPU que atualmente é atribuída à sessão grauita do Colab (T4) possui memória mais que o suficiente para leitura de todos os dados desse dataset

In [ ]:
%time
base = cudf.read_csv('/content/dataset/US_Accidents_March23.csv')

In [ ]:
base.shape

## 2) Remoção de valores nulos

In [ ]:
df = base.dropna()

In [ ]:
df

In [ ]:
type(df)

## 3) Remoção de colunas desnecessárias

In [ ]:
cols_dataset = df.columns.values.tolist()
cols_dataset

In [ ]:
cols = ['Severity', 'Source', 'County', 'State', 'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']
cols

In [ ]:
len(cols)

In [ ]:
drop_cols = [c for c in cols_dataset if c not in cols]
drop_cols

In [ ]:
df = df.drop(columns=drop_cols)

In [ ]:
df

## 4) Tratamento com Label encoder  



In [ ]:
from cuml.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()



Faremos de um jeito melhor, ao invés de escrever manualmente para cada uma das 21 colunas (lembrando que o Severity ficará de fora), como abaixo  

```
df['Weather_Condition'] = label_encoder.fit_transform(df['Weather_Condition'])
df['Amenity']= label_encoder.fit_transform(df['Amenity'])
[...]
```

In [ ]:
for c in cols:
  if c != 'Severity':
    df[c] = label_encoder.fit_transform(df[c])

In [ ]:
df

## 5) Balanceamento do dataset


Podemos checar como os valores estão distribuídos nas diferentes categorias usando o value_counts()

In [ ]:
df['Severity'].value_counts()

In [ ]:
df = df.to_pandas()

In [ ]:
type(df)

In [ ]:
from sklearn.utils import resample

In [ ]:
df_s1 = df[df['Severity'] == 1]
df_s2 = df[df['Severity'] == 2]
df_s3 = df[df['Severity'] == 3]
df_s4 = df[df['Severity'] == 4]

In [ ]:
df_s1

In [ ]:
count = int(max(df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0]) / 20)

In [ ]:
count

In [ ]:
df_s1 = resample(df_s1, replace=df_s1.count()[0] < count, n_samples=count, random_state=42)
df_s2 = resample(df_s2, replace=df_s2.count()[0] < count, n_samples=count, random_state=42)
df_s3 = resample(df_s3, replace=df_s3.count()[0] < count, n_samples=count, random_state=42)
df_s4 = resample(df_s4, replace=df_s4.count()[0] < count, n_samples=count, random_state=42)

In [ ]:
df = pd.concat([df_s1, df_s2, df_s3, df_s4])

In [ ]:
print(df['Severity'].value_counts())

#df.groupby(by='Severity')['Severity'].count()

## 6) Tratamento com o OneHot Encoding

In [ ]:
X = df.drop('Severity', axis=1)

In [ ]:
X

In [ ]:
from cuml.preprocessing import OneHotEncoder
from cuml.compose import ColumnTransformer

In [ ]:
idx = [*range(0,20)]
idx

In [ ]:
onehotencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(sparse=False), idx)], remainder='passthrough')

In [ ]:
X = onehotencoder.fit_transform(X)

In [ ]:
X.shape

## 7) Escalonamento dos valores

In [ ]:
from cuml.preprocessing import StandardScaler
scaler_dataset = StandardScaler()

X = scaler_dataset.fit_transform(X)

In [ ]:
print(X)
print(X.shape)

## 8) Divisão entre previsores e classe

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = df['Severity']

y = LabelEncoder().fit_transform(y)

In [ ]:
y

In [ ]:
y.shape

In [ ]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.15, random_state = 42)

In [ ]:
X_treinamento.shape, X_teste.shape, y_treinamento.shape, y_teste.shape

In [ ]:
import pickle
with open('dataset.pkl', mode = 'wb') as f:
  pickle.dump([X_treinamento, y_treinamento, X_teste, y_teste], f)

## 9) Treinamento com o algoritmo

In [ ]:
from cuml.neighbors import KNeighborsClassifier

In [ ]:
with open('dataset.pkl', 'rb') as f:
  X_treinamento, y_treinamento, X_teste, y_teste = pickle.load(f)

In [ ]:
X_treinamento_cudf = cudf.DataFrame.from_pandas(X_treinamento)
X_teste_cudf = cudf.DataFrame.from_pandas(X_teste)

y_treinamento_cudf = cudf.Series(y_treinamento.values)
y_teste_cudf = cudf.Series(y_teste.values)

In [ ]:
%time

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_treinamento, y_treinamento)

## 10) Predição e cálculo de acurácia

In [ ]:
previsoes = knn.predict(X_teste)
previsoes

In [ ]:
from cuml.metrics import accuracy_score

In [ ]:
accuracy_score(y_teste, previsoes)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!nvidia-smi